# **Data Cleaning for Primary Dataset ("Hydra-Movie-Scrape.csv")**

**Import CSV and apply encoding to handle characters from multiple languages in dataset**

In [2]:
import numpy as np
import pandas as pd
import chardet
import scipy.stats as stats

# Load the dataset
file_path = 'Hydra-Movie-Scrape.csv'

# Read file in binary to detect the encoding
with open(file_path, 'rb') as file:
    raw_data = file.read(100000)  # Read only the first 100,000 bytes or we risk exceeding data rate limit

# Detect the encoding used
result = chardet.detect(raw_data)
encoding = result['encoding']

# Load the CSV file and apply the detected encoding
df = pd.read_csv(file_path, encoding=encoding)

# Examine the dataset to identify columns with missing values and outliers.
# Display the first few rows of the pandas dataframe
print(df.head(15))


                                                Title  Year  \
0                         Patton Oswalt: Annihilation  2017   
1                                       New York Doll  2005   
2   Mickey's Magical Christmas: Snowed in at the H...  2001   
3                          Mickey's House of Villains  2001   
4                                       And Then I Go  2017   
5                            An Extremely Goofy Movie  2000   
6                                        Peter Rabbit  2018   
7                                          Love Songs  2007   
8                                                  89  2017   
9                                      The Foster Boy  2011   
10                                    Forever My Girl  2018   
11                            Tom Segura: Disgraceful  2018   
12      The Secret Rules of Modern Living: Algorithms  2015   
13                                Secrets in the Fall  2015   
14                                       Silent Night  

In [3]:
# Display info about the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3940 entries, 0 to 3939
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Title            3940 non-null   object 
 1   Year             3940 non-null   int64  
 2   Summary          3935 non-null   object 
 3   Short Summary    3939 non-null   object 
 4   Genres           3940 non-null   object 
 5   IMDB ID          3940 non-null   object 
 6   Runtime          3940 non-null   int64  
 7   YouTube Trailer  3893 non-null   object 
 8   Rating           3940 non-null   float64
 9   Movie Poster     3940 non-null   object 
 10  Director         3940 non-null   object 
 11  Writers          3922 non-null   object 
 12  Cast             3916 non-null   object 
dtypes: float64(1), int64(2), object(10)
memory usage: 400.3+ KB


**Handle Missing Data**

In [4]:
# Examing missing values:

# Print missing values percentages:
missing_ratio = df.isnull().mean()
print("\nPercentage Missing Values:")
print(missing_ratio)


Percentage Missing Values:
Title              0.000000
Year               0.000000
Summary            0.001269
Short Summary      0.000254
Genres             0.000000
IMDB ID            0.000000
Runtime            0.000000
YouTube Trailer    0.011929
Rating             0.000000
Movie Poster       0.000000
Director           0.000000
Writers            0.004569
Cast               0.006091
dtype: float64


> <ins>Note:</ins> Since there are virutally no Nan's in the dataset we will move on.

**Handle Duplicates**

In [5]:
# Check if duplicates exist

# Identify duplicate rows
duplicates = df.duplicated()

# Count the number of duplicates
num_duplicates = duplicates.sum()

# Display the number of duplicates
print(f'Number of duplicate rows: {num_duplicates}')

Number of duplicate rows: 0


> <ins>Note:</ins> Since there are no duplicates, we will move on.

**Handle Outliers**

In [6]:
# Set the global display format for floating-point numbers
pd.options.display.float_format = '{:.2f}'.format

# Examine the distributions
print("\nData Distributions:")
print(df.describe())
print("\nObjects-Categorical Variables:")
print(df.describe(include=['object','string']))



Data Distributions:
         Year  Runtime  Rating
count 3940.00  3940.00 3940.00
mean  2012.13   100.35    6.56
std      4.82    29.80    0.89
min   2000.00     0.00    1.70
25%   2009.00    91.00    6.10
50%   2013.00   102.00    6.60
75%   2016.00   115.00    7.10
max   2018.00   338.00    9.50

Objects-Categorical Variables:
           Title                                            Summary  \
count       3940                                               3935   
unique      3927                                               3935   
top     The Gift  Patton Oswald, despite a personal tragedy, pro...   
freq           2                                                  1   

       Short Summary Genres    IMDB ID YouTube Trailer  \
count           3939   3940       3940            3893   
unique          3938    836       3940            3883   
top     Add a Plot »  Drama  tt7026230     UKY3scPIMd8   
freq               2    116          1               9   

                     

> <ins>Note:</ins> Descriptive Stats shows us that the Min value for Runtime is 0 which should not be the case. Further
investigation shows 193 records where Runtime = 0 :

In [7]:
# Outlier Work

# Ensure the 'Runtime' column is numeric
df['Runtime'] = pd.to_numeric(df['Runtime'], errors='coerce')

# Calculate the number of records where Runtime is 0
num_runtime_zero = (df['Runtime'] == 0).sum()

# Display the number of records with Runtime = 0
print(f'The number of records where Runtime is 0: {num_runtime_zero}')


The number of records where Runtime is 0: 193


**Use Imputation to update those records with Average Runtime instead:**

In [17]:
# First we calculate the average runtime of non-zero values
#average_runtime_non_zero = df[df['Runtime'] != 0]['Runtime'].mean()

# Since Runtime is an int, calculate the average runtime of non-zero values and round to the nearest integer
average_runtime_non_zero = round(df[df['Runtime'] != 0]['Runtime'].mean())

# Display the rounded average runtime
print(f'The rounded average runtime of all non-zero movies is: {average_runtime_non_zero} minutes')


The rounded average runtime of all non-zero movies is: 106 minutes


In [19]:
# Then we Impute the zero Runtime values with the average Runtime value:
df.loc[df['Runtime'] == 0, 'Runtime'] = average_runtime_non_zero

# Verify the update
num_runtime_zero_after_imputation = (df['Runtime'] == 0).sum()

# Print verification results
print(f'The number of records where Runtime is 0 after imputation: {num_runtime_zero_after_imputation}')
print(f'The average runtime of all movies is now: {df["Runtime"].mean()} minutes')

The number of records where Runtime is 0 after imputation: 0
The average runtime of all movies is now: 105.49695431472081 minutes


In [20]:
# force the imputed values to go into Runtime as int:
df['Runtime'] = df['Runtime'].astype(int)

**Write cleaned data to an output CSV file:**

In [21]:
# Write the cleaned data to an output CSV file
output_file_path = 'Cleaned-Movies-Primary.csv'
df.to_csv(output_file_path, index=False)

print(f'The cleaned data has been written to {output_file_path}')

The cleaned data has been written to Cleaned-Movies-Primary.csv
